## 1. Setup StockX

In [1]:
# In your Jupyter notebook (TEST_Daily_Analysis.ipynb)
import sys
from pathlib import Path

# Add the project root directory to Python path
ROOT_DIR = Path.cwd().parent  # Goes up one level from 'scanner' to 'stockbot4'
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))


from ib_insync import *
util.startLoop()
import stock

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=15)
sx  = stock.StockX(ib, 'AKBA')
spy = stock.StockX(ib, 'SPY')

In [2]:
spy.RUN_DAILY(isMarket=True)

sx.set_ls('LONG')
sx.RUN_DAILY(spy)

-------------------------------------------------------------------------------------------------
get_hist_data            : SPY 1 day (file_interval='1_day')
Data Path                : C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store/SPY_1_day.csv
Data Stored    - Start   : 2023-12-07 00:00:00, End: 2025-03-20 00:00:00, Rows: 321
Data Requested - Start   : 2024-03-18 00:00:01, End: now
Missing dates            : 1
Getting missing data     : [('2025-03-17', '2025-03-20')]
-------------------------------------------------------------------------------------------------
StockX::setup_frame: SPY 1 day ohlcv 52 weeksAgo now force_download=False : (252, 5)
-------------------------------------------------------------------------------------------------
get_hist_data            : SPY 1 hour (file_interval='1_hour')
Data Path                : C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store/SPY_1_hour.csv
Data Stored    - Start   : 2024-02-16 04:00:

In [3]:
sx.ls
sx.score_cols.__dict__

{'vol_TODC': 'VOL_TODC_10',
 'D1': 'Score_LONG_Daily_VALID_Score',
 'M5': '',
 'M2': '',
 'M1': '',
 'time': '',
 'level_premkt_gt': '',
 'touches': '',
 'reset_if_breaks': '',
 'pullback': '',
 'retest': '',
 'bsw': '',
 'buysetup': '',
 'rtm': 'Score_LONG_RTM',
 'buy': '',
 'gaps': 'Score_LONG_Gaps'}

In [4]:
sx.get_frame_data('1 hour').columns

Index(['open', 'high', 'low', 'close', 'volume', 'SPY_volume', 'SPY_close',
       'SPY_open', 'SPY_high', 'SPY_low', 'MA_cl_50', 'MA_cl_150', 'MA_cl_200',
       'ATR_14', 'HP_hi_3', 'LP_lo_3', 'HP_hi_10', 'LP_lo_10', 'Res_1',
       'Res_1_Upper', 'Res_1_Lower', 'Res_2', 'Res_2_Upper', 'Res_2_Lower',
       'Sup_1', 'Sup_1_Upper', 'Sup_1_Lower', 'Sup_2', 'Sup_2_Upper',
       'Sup_2_Lower', 'VWAP_session', 'VOL_Acum', 'pre_mkt_high',
       'pre_mkt_low', 'intraday_high_9.35', 'intraday_low_9.35',
       'intraday_high', 'intraday_low', 'prev_day_high', 'prev_day_low',
       '1 day_Res_1_Lower', '1 day_Sup_1_Upper', '1 day_MA_cl_200',
       '1 day_MA_cl_50', 'VOL_TODC_10'],
      dtype='object')

In [5]:
sx.set_daily_stats(printStats=True)

Using cached data for AKBA (age: 2025-03-19 17:43:25)


,symbol,ls,snapshotTime,status,price,marketCap,currentPrice,peRatio,vol10dayAvg,vol_TODC,...,breaksAbove10DayAvg,breaksAbove50MA,breaksAbove200MA,breaksBelow50MA,breaksBelow200MA,scoreVol,scoreGaps,scoreRTM,score_1D,validAv_1D
0,AKBA,LONG,2025-03-20 20:10:56,DAILY_TA,2.05,602.3892,2.55,-99999.99,4.7809,0.0,...,False,False,False,False,False,125.8,20.0,11.9,52.57,0.0


In [6]:
sx.get_frame_data('1 day', colsContains='RTM').tail(10)

,RTM_L_Res_1_Lower,VAL_L_RTM_L_Res_1_Lower_>_50,Score_LONG_RTM
date,,,
2025-03-07,16.129032,0.0,16.1
2025-03-10,24.731183,0.0,24.7
2025-03-11,9.603659,0.0,9.6
2025-03-12,9.790210,0.0,9.8
2025-03-13,4.854369,0.0,4.9
2025-03-14,22.119233,0.0,22.1
2025-03-17,1.373357,0.0,1.4
2025-03-18,100.000000,100.0,100.0
2025-03-19,100.000000,100.0,100.0


In [7]:
h = 2.29
l = 1.755
rng = h-l
atr = 0.135
val = 1.75



top=0.0150 
bot=0.007500 
body=0.2449

bar_val = (top - bot + body) 

# 2025-03-14 00:00:00 open: 1.77, close: 2.26, score: 1.751 (top=0.0150 - bot=0.007500 + body=0.2449) / (high=2.29 - low=1.755)

print(rng)
rng / atr
bar_val

0.5350000000000001


0.2524

In [8]:
2.26-1.77

0.48999999999999977

In [9]:
sx.get_frame('1 day').plot()

In [9]:
vol_pct_chg = sx.get_frame_data('1 hour')['VOL_TODC_10'].iat[-1]


print(f'Volume percent change: {vol_pct_chg:.2f}%')

Volume percent change: 125.80%


## 2. Fundamentals

In [10]:
sx.RUN_FUNDAMENTALS(allowedETFs=['XLY', 'XLK', 'XLC'], maxDaysOld=10)

## 3. Setup

In [11]:
sx.RUN_SETUP(spy, dataType='ohlcv', forceDownload=False, endDate='2025-02-20')

-------------------------------------------------------------------------------------------------
get_hist_data            : AKBA 1 day (file_interval='1_day')
Data Path                : C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store/AKBA_1_day.csv
Data Stored    - Start   : 2024-01-02 00:00:00, End: 2025-03-20 00:00:00, Rows: 305
Data Requested - Start   : 2024-02-19 00:00:01, End: 2025-02-20
Missing dates            : 0
Getting missing data     : False
-------------------------------------------------------------------------------------------------
StockX::setup_frame: AKBA 1 day ohlcv 52 weeksAgo 2025-02-20 force_download=False : (252, 5)
-------------------------------------------------------------------------------------------------
get_hist_data            : AKBA 1 hour (file_interval='1_hour')
Data Path                : C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store/AKBA_1_hour.csv
Data Stored    - Start   : 2025-02-18 07:00:00, En

## 4. PreMarket

In [12]:
sx.RUN_PRE_MARKET('LONG', timeToRun='09:00', minPreMarketScore=50, displayCharts=True, printStats=True) # runs daily and premarket volume analysis

AttributeError: 'StockX' object has no attribute 'setup_TA_PreMarket'

In [ ]:
sx.display_columns('1 day')
sx.display_columns('1 day', contains='SPY')

## 5. Intraday

In [ ]:
sx.RUN_INTRADAY(updateEverySeconds=60, myTradingTimes=[('9:35', '15:00')], maxTrades=5, displayCharts=True, logTrades=True)

In [ ]:
import pandas as pd
import strategies.preset_strats as ps
import strategies.signals as sig
import strategies.ta as ta
from typing import Dict, Any, List
from chart.chart import ChartArgs

sx.import_all_market_data(spy)
sx.set_ls('LONG')
sx.setup_TA_1D(lookBack=25, atrSpan=14, sigRow=3, validationRow=4)

if sx.day_score_passed(minScore=50):
    sx.setup_TA_intraday(lookBack=200, atrSpan=50, sigRow=3, validationRow=4)
    
    if sx.pre_market_passed(minScore=50):
        sx.RUN(updateEvery=30, myTradingTimes=[('9:35', '15:00')], maxTrades=5, displayCharts=False, logTrades=True)




spy.frames['1 day'].plot()
sx.frames['1 day'].plot()
sx.frames['1 hour'].plot()
sx.frames['5 mins'].plot()

In [ ]:
sx.display_columns('5 mins', contains='PB')

In [ ]:
dfx = sx.display_columns('5 mins', contains='PB')
dfx.loc['2025-02-20 09:35:00':].head(50)

 # StockX Score

In [ ]:
ib.disconnect()

In [ ]:
from dataclasses import dataclass

